In [1]:
# 1. Install Gradio
!pip install gradio --quiet

# 2. Imports
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 3. Load and preprocess the dataset
def load_data():
    df = pd.read_csv("/content/insurance_customer_data (1).csv")
    features = df[['Age', 'Annual_Premium', 'Vintage']]
    target = df['Response']
    le = LabelEncoder()
    y_encoded = le.fit_transform(target)
    return features, y_encoded, le

# 4. Train Self-Training model
def train_self_training(X, y):
    # Split into labeled and unlabeled
    X_labeled, X_unlabeled, y_labeled, _ = train_test_split(
        X, y, train_size=0.2, stratify=y, random_state=42
    )

    y_combined = np.concatenate([y_labeled, [-1]*len(X_unlabeled)])
    X_combined = pd.concat([X_labeled, X_unlabeled])

    base_clf = LogisticRegression(max_iter=1000)
    self_training = SelfTrainingClassifier(base_clf, threshold=0.8, verbose=False)
    self_training.fit(X_combined, y_combined)

    return self_training

# Load and train model initially
X_full, y_full, label_encoder = load_data()
model = train_self_training(X_full, y_full)

# 5. Prediction function
def predict(age, premium, vintage):
    input_data = pd.DataFrame([[age, premium, vintage]], columns=['Age', 'Annual_Premium', 'Vintage'])
    pred = model.predict(input_data)[0]
    label = label_encoder.inverse_transform([pred])[0]
    return f"🧠 Predicted Response: {label}"

# 6. Retrain function (simulate self-learning loop)
def retrain_model():
    global model
    model = train_self_training(X_full, y_full)
    return "✅ Model retrained with self-labeled data."

# 7. Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🛡️ Insurance Customer Prediction (Self-Learning ML)")
    with gr.Row():
        age = gr.Number(label="Customer Age", value=35)
        premium = gr.Number(label="Annual Premium", value=30000)
        vintage = gr.Number(label="Vintage (Days as Customer)", value=100)

    predict_btn = gr.Button("🔍 Predict Response")
    output = gr.Textbox(label="Prediction Result")

    retrain_btn = gr.Button("🔁 Retrain Model")

    predict_btn.click(predict, inputs=[age, premium, vintage], outputs=output)
    retrain_btn.click(retrain_model, outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d4f829cc1e0ca63d9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
